In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from numpy import NaN
import warnings
warnings.filterwarnings("ignore")

In [2]:
X = '000300.XSHG'
type = 'IF'
start_date = '2020-02-22'
end_date = '2021-07-16'
period = get_trading_dates(start_date,end_date)
future_list = ['IF2003','IF2004','IF2005','IF2006','IF2007','IF2008','IF2009',
               'IF2010','IF2011','IF2012','IF2101','IF2102','IF2103','IF2104',
               'IF2105','IF2106','IF2107','IF2108','IF2109']
df = pd.DataFrame(index = period,columns = future_list)
df_return = get_price(future_list,start_date,end_date,fields = 'close',expect_df = False)
df_return = df_return[future_list]
df_X = get_price(X,start_date,end_date,'1d',fields = 'close',expect_df = False)
for i in range(df_return.shape[0]):
    df_return.iloc[i,:] = df_X[i]-df_return.iloc[i,:]

df_return_daily = df_return - df_return.shift(-1)

In [3]:
'''策略一：始终购买主力合约'''
Futures = futures.get_dominant(type,start_date,end_date,rule = 1,rank = 1)
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        func = lambda x: 1 if (Futures[i] == df.columns[j]) else 0
        df.iloc[i,j] = func(df.iloc[i,j])
sum_return = 0
for i in range(0,len(Futures)-1):
    sum_return += (df_return_daily[Futures[i]][i])

sum_return

415.46610000000055

In [4]:
'''策略二：始终购买次主力合约'''
Futures = futures.get_dominant(type,start_date,end_date,rule = 1,rank = 2)
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        func = lambda x: 1 if (Futures[i] == df.columns[j]) else 0
        df.iloc[i,j] = func(df.iloc[i,j])
sum_return = 0
for i in range(0,len(Futures)-1):
    sum_return += df_return_daily[Futures[i]][i]

sum_return

230.66610000000264

In [5]:
'''策略三：始终购买近月合约'''
maturity_list = []
for f in future_list:
    maturity_list.append(instruments(f).maturity_date)
sum_return = sum(df_return_daily.loc[:maturity_list[0]].iloc[:-1,0])
for i in range(1,len(maturity_list)):
    sum_return += (sum(df_return_daily.loc[maturity_list[i-1]:maturity_list[i]].iloc[:-1,i]))
sum_return

376.06610000000273

In [6]:
'''策略四：始终购买远月合约'''
maturity_list = []
for f in future_list:
    maturity_list.append(instruments(f).maturity_date)
sum_return = sum(df_return_daily.loc[:maturity_list[1]].iloc[:-1,1])
for i in range(1,len(maturity_list)-1):
    sum_return += (sum(df_return_daily.loc[maturity_list[i]:maturity_list[i+1]].iloc[:-1,i+1]))
sum_return

371.66610000000264

In [16]:
'''策略五：购买当日bpd小者'''
basis_df = pd.DataFrame(index = get_trading_dates(start_date,end_date),columns = ['dom1','dom2', X, 'dom1_close','dom2_close','basis1',
                                                                            'basis2','matu1','matu2','bpd1','bpd2'])
basis_df['dom1'] = futures.get_dominant(type,start_date, end_date, rule = 1, rank = 1)
basis_df['dom2'] = futures.get_dominant(type,start_date,end_date, rule = 1, rank = 2)

for i in range(basis_df.shape[0]):
    basis_df['dom1_close'][i] = float(get_price(basis_df['dom1'][i],basis_df.index[i],basis_df.index[i],fields = 'close').iloc[0])
    basis_df['dom2_close'][i] = float(get_price(basis_df['dom2'][i],basis_df.index[i],basis_df.index[i],fields = 'close').iloc[0])
                         
basis_df[X] = get_price(X, start_date, end_date, frequency = '1d', fields = ['close'],expect_df = False)
basis_df['basis1'] = basis_df[X]-basis_df['dom1_close']
basis_df['basis2'] = basis_df[X]-basis_df['dom2_close']
basis_df['matu1'] = basis_df['dom1'].apply(lambda x:instruments(x).maturity_date)
basis_df['matu2'] = basis_df['dom2'].apply(lambda x:instruments(x).maturity_date)
for i in range(basis_df.shape[0]):
    basis_df['bpd1'][i] = len(get_trading_dates(basis_df.index[i],basis_df.matu1[i]))
    basis_df['bpd2'][i] = len(get_trading_dates(basis_df.index[i],basis_df.matu2[i]))
basis_df['bpd1'] = basis_df['basis1'] / basis_df['bpd1']
basis_df['bpd2'] = basis_df['basis2'] / basis_df['bpd2']

Futures = []
for l in range(basis_df.shape[0]):
    if (basis_df.bpd1[l] <= basis_df.bpd2[l]):
        Futures.append(basis_df.dom1[l])
    else:
        Futures.append(basis_df.dom2[l])
        
sum_return = 0
for i in range(0,len(Futures)-1):
    sum_return += df_return_daily[Futures[i]][i]

sum_return

200.86610000000837

In [8]:
# basis的绝对值和第二天return的相关性 当basis比较大的时候 就算bpd小 第二天也会亏掉很多 可以把大basis滤掉
zibianliang = basis_df.basis1

In [9]:
'''策略六：如果basis向下冲破5日basis_ma则买2'''
def generate_ma(input = pd.DataFrame(),term = 5):
    input['basis_diff'] = input['basis1']-input['basis2']
    input['basis_diff_ma'] = input['basis_diff'].rolling(window =term).mean()
    return input

def go_ma(input = pd.DataFrame()):
    Futures = []
    length = input.shape[0]
    selection = pd.DataFrame(index = input.index, columns = ['basis1(y/n)','basis2(y/n)','chg1','chg2'])
    selection['basis1(y/n)'] = 1 #默认都选主力
    for i in range(length):
        if input['basis_diff'][i]<input['basis_diff_ma'][i]:
            Futures.append(input.dom2[i])
        else:
            Futures.append(input.dom1[i])
    return Futures

generate_ma(basis_df)
Futures = go_ma(basis_df)
sum_return = 0
for i in range(0,len(Futures)-1):
    sum_return += df_return_daily[Futures[i]][i]

sum_return

345.2661000000062

In [17]:
df_return_daily.index[18]
df_return_daily[Futures[18]][18]


-1.931300000000192